# Preprocessing Data

Here I am preprocessing the data for the audio classifier. 

It includes:

- Converting the midi files from https://jazzomat.hfm-weimar.de/dbformat/dbcontent.html into WAV files. 
- Sorting the files into folders for each artist. 
- Splitting the files into 5 second segments and saving them with the same names in the same respective sub folders. 

I had lots of help writing this code from Louie and LLM models. 

Code: 
- https://github.com/JinayJain/timidity
- https://stackoverflow.com/questions/60105626/split-audio-on-timestamps-librosa 

DUE TO FOLDER SIZE CONSTRAINTS PLEASE FIND AND DOWNLOAD THE DATA FILES HERE:
https://drive.google.com/drive/folders/1q7doAYmSEUWp3-QPFnOoosSbLZHpmVnC?usp=sharing 

#### Converting MIDI files to WAV

In [ ]:
# Code from https://github.com/JinayJain/timidity and help from Louis! 
# Converting MIDI files into WAV files. 
# https://stackoverflow.com/questions/10989005/do-i-understand-os-walk-right
from timidity import Parser, play_notes
from scipy.signal import square, sawtooth
import numpy as np
from scipy.io import wavfile
import os.path

for path, directories, files in os.walk('/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/jazz_solos'):
     for file in files:
        try:
            midi_file_path = os.path.join(path, file)
            ps = Parser(midi_file_path)
            audio, player = play_notes(*ps.parse(), np.sin, wait_done=False)
            wavfile.write(f"{file}.wav", 44100, audio)
            # player.wait_done()
        except Exception as e: 
            print(e) 



#### Moving files into class folders

In [ ]:
# This code was written by ChatGTP. Due to how much time I had already spent sourcing this data I didn't want to waste more 
# working out how to do this! 

import os
import shutil

def move_wav_files(source_folder, destination_folder):
    # Get a list of all files in the source folder
    files = os.listdir(source_folder)

    # Create a dictionary to store files based on common prefixes
    file_dict = {}

    # Iterate through each file in the source folder
    for file in files:
        if file.endswith(".wav"):
            # Extract the common prefix up to the underscore
            prefix = file.split("_")[0]

            # If the prefix is not in the dictionary, create a new list for it
            if prefix not in file_dict:
                file_dict[prefix] = []

            # Add the file to the list associated with its prefix
            file_dict[prefix].append(file)

    # Iterate through the dictionary and move files to new folders
    for prefix, files_list in file_dict.items():
        folder_path = os.path.join(destination_folder, prefix)

        # Create the folder if it doesn't exist
        os.makedirs(folder_path, exist_ok=True)

        # Move each file to its respective folder
        for file in files_list:
            source_path = os.path.join(source_folder, file)
            destination_path = os.path.join(folder_path, file)
            shutil.move(source_path, destination_path)
            print(f"Moved '{file}' to '{folder_path}'")

# Example usage:
source_folder = "/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/wav_files"
destination_folder = "/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/wav_folders"

move_wav_files(source_folder, destination_folder)


In [ ]:
# @Book{Pfleiderer:2017:BOOK,
#   title =     {{I}nside the {J}azzomat - {N}ew {P}erspectives for {J}azz  {R}esearch},
#   publisher = {Schott Campus},
#   year =      {2017},
#   editor =    {Pfleiderer, Martin and Frieler, Klaus and Abe{\ss}er, Jakob and Zaddach, Wolf-Georg and Burkhart, Benjamin},
# }

In [ ]:
from pydub import AudioSegment
import os
from pydub import AudioSegment
import imageio_ffmpeg as ffmpeg
import librosa 

# Code is from https://stackoverflow.com/questions/60105626/split-audio-on-timestamps-librosa 

# First load the file
audio, sr = librosa.load('/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/jazz_solos/Art_Pepper/ArtPepper_Anthropology_FINAL.wav')
# Get number of samples for 2 seconds; replace 2 by any number
buffer = 30 * sr

samples_total = len(audio)
samples_wrote = 0
counter = 1
while samples_wrote < samples_total:
    #check if the buffer is not exceeding total samples 
    if buffer > (samples_total - samples_wrote):
        buffer = samples_total - samples_wrote
    block = audio[samples_wrote : (samples_wrote + buffer)]
    out_filename = "split_" + str(counter) + "_" + file_name
    # Write 2 second segment
    librosa.soundfile.write(out_filename, block, sr)
    counter += 1
    samples_wrote += buffer



### Splitting the data into 5 second splits

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import os 
import os, sys


window_size = 1024 

# This code is from https://stackoverflow.com/questions/60105626/split-audio-on-timestamps-librosa
# With help from ChatGTP 
def split_and_save(audio_path, output_folder, duration=30):
    audio_data, sr = librosa.load(audio_path, sr=None)
    total_segments = int(np.ceil(len(audio_data) / (duration * sr)))

    for i in range(total_segments):
        start_sample = i * duration * sr
        end_sample = min((i + 1) * duration * sr, len(audio_data))
        segment = audio_data[start_sample:end_sample]
        windows = librosa.util.frame(segment, frame_length=window_size)
        windows = windows.transpose()
        amps = []
        for buffer in windows:
            mean = np.abs(buffer.mean())
            amps.append(mean)
        plt.plot(amps)

        # Save the segment with the same filename and a segment number
        filename = os.path.join(output_folder, f"{os.path.basename(audio_path)}_{i + 1}.wav")
        librosa.output.write_wav(filename, segment, sr)

input_folder = "/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/Jazz_solo_testing_data"
output_folder = "/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/Jazz_solo_testing_data_split"

for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith(".wav"): 
            audio_path = os.path.join(root, file)
            split_and_save(audio_path, output_folder, duration=30)


#### Features
This code extracts the features of the audio files using librose. I was going to use this to build a simpiler classification model and compare however due to time constraints I didn't make it this far. 

In [ ]:
# Code written with help from Louie 

#This is the label you learn (added as last feature, split off to use a y, other features are X)

window_seconds = 5
features = []
key = [[]]
label = 0
for folder, directory, files in os.walk('/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/wav_folders'):
    artist = os.path.basename(folder)
    for i in files:
        file_path = os.path.join(folder, i)
        audio_data, sr = librosa.load(file_path)
        window_size = sr*window_seconds
        windows = librosa.util.frame(audio_data, frame_length=window_size, hop_length=window_size)
        windows = windows.transpose()
        for buffer in windows:
            chroma = librosa.feature.chroma_stft(y = buffer)
            chroma = list(chroma.mean(axis=1))
            chroma.append((buffer**2).mean())
            features.append(chroma + [label])
    
    key.append(artist + [label])
    label += 1

features = np.array(features)
features.shape
print(key)

### Removing files that aren't the same length

This code was written with help LLMs

In [ ]:
import os
import librosa

def remove_short_files(root_dir):
    #Walking through the folder
    for root, _, files in os.walk(root_dir):  
        for file in files:
            #Find the audio files
            if file.endswith(".wav"): 
                file_path = os.path.join(root, file)
                duration = librosa.get_duration(filename=file_path) 
                if duration < 5: 
                        # Remove them if they aren't 5 seconds long 
                        os.remove(file_path)  

# Example usage:
root_dir = "/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/wav_folders_split"  
remove_short_files(root_dir)

### Create Spectograms

This code is from https://github.com/nicknochnack/DeepAudioClassification

In [ ]:
import os
import torchaudio
import matplotlib.pyplot as plt

SPECTROGRAM_DPI = 90
DEFAULT_SAMPLE_RATE = 44100
DEFAULT_HOPE_LENGTH = 1024

class Audio:
    def __init__(self, filepath, hop_length=DEFAULT_HOPE_LENGTH, sample_rate=DEFAULT_SAMPLE_RATE):
        self.hop_length = hop_length
        self.sample_rate = sample_rate
        self.waveform, self.sample_rate = torchaudio.load(filepath)

    def plot_spectrogram(self):
        waveform = self.waveform.numpy()
        _, axes = plt.subplots(1, 1)
        axes.specgram(waveform[0], Fs=self.sample_rate)
        plt.axis('off')
        plt.show(block=False)
    
    def write_disk_spectrogram(self, path, dpi=SPECTROGRAM_DPI):
        self.plot_spectrogram()
        plt.savefig(path, dpi=dpi, bbox_inches='tight')
        plt.close()

input_folder = "/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/wav_folders_split"
output_folder = "/Users/loiskelly/Documents/GitHub/ai4mediaproject/data/wav_split_specto"

for folder, _, files in os.walk(input_folder):
    for i, file in enumerate(files):
        file_path = os.path.join(folder, file)
        artist = os.path.basename(folder)
        output_path = os.path.join(output_folder, f"{artist}_{i + 1}.png")
        
        # Create the output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)
        
        sound = Audio(file_path)
        sound.write_disk_spectrogram(output_path)
